In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import xgboost as xgb
import lightgbm as lgb
import catboost as cb

from sklearn import metrics
from sklearn.model_selection import train_test_split, GridSearchCV

In [3]:
path = "/kaggle/input/predict-student-performance-from-game-play/"

use_cols = ["session_id","elapsed_time","event_name","name","level","room_coor_x","room_coor_y",
            "screen_coor_x","screen_coor_y","hover_duration","level_group"]
dtypes = {"session_id":"object","elapsed_time":"int64","event_name":"object","name":"object",
          "level":"object","room_coor_x":"float64","room_coor_y":"float64",
          "screen_coor_x":"float64","screen_coor_y":"float64","hover_duration":"float64",
          "level_group":"object"}

train_df = pd.read_csv(path + "train.csv",
                      dtype=dtypes,
                      usecols=use_cols)

In [ ]:
columns = ["session_id", "index", "elapsed_time", "event_name", "name ", "level ", "page ", "room_coor_x", "room_coor_y", "screen_coor_x",
          "screen_coor_y", "hover_duration", "text", "fqid", "room_fqid", "fullscreen", "hq", "music", "level_group"]

In [4]:
train_df.head()

,session_id,elapsed_time,event_name,name,level,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,level_group
0,20090312431273200,0,cutscene_click,basic,0,-413.991405,-159.314686,380.0,494.0,NaN,0-4
1,20090312431273200,1323,person_click,basic,0,-413.991405,-159.314686,380.0,494.0,NaN,0-4
2,20090312431273200,831,person_click,basic,0,-413.991405,-159.314686,380.0,494.0,NaN,0-4
3,20090312431273200,1147,person_click,basic,0,-413.991405,-159.314686,380.0,494.0,NaN,0-4
4,20090312431273200,1863,person_click,basic,0,-412.991405,-159.314686,381.0,494.0,NaN,0-4


In [5]:
test_df = pd.read_csv(path + "test.csv")
test_df.head()

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,...,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group,session_level
0,20090109393214576,0,0,cutscene_click,basic,0,NaN,-413.991405,75.685314,380.0,...,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,0,0,1,0-4,NaN
1,20090109393214576,1,1965,person_click,basic,0,NaN,-105.991405,-63.314686,688.0,...,NaN,"Whatcha doing over there, Jo?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4,NaN
2,20090109393214576,2,3614,person_click,basic,0,NaN,-418.991405,47.685314,375.0,...,NaN,Just talking to Teddy.,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4,NaN
3,20090109393214576,3,5330,person_click,basic,0,NaN,-110.991405,-57.314686,683.0,...,NaN,I gotta run to my meeting!,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4,NaN
4,20090109393214576,4,6397,person_click,basic,0,NaN,-110.991405,-57.314686,683.0,...,NaN,"Can I come, Gramps?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4,NaN


In [7]:
sub = pd.read_csv(path + "sample_submission.csv")
sub.head()

,session_id,correct,session_level
0,20090109393214576_q1,0,NaN
1,20090312143683264_q1,0,NaN
2,20090312331414616_q1,0,NaN
3,20090109393214576_q2,0,NaN
4,20090312143683264_q2,0,NaN


In [8]:
train_labels = pd.read_csv(path + "train_labels.csv")
train_labels.head()

,session_id,correct
0,20090312431273200_q1,1
1,20090312433251036_q1,0
2,20090312455206810_q1,1
3,20090313091715820_q1,0
4,20090313571836404_q1,1
